# Geocoding addresses with a script

We have a list of payday lender addresses that we need to also get latitude and longitude for. While services do exist that will turn address data into coordinates, we can use a third-party Python library to bounce address information off a range of geocoding services and get back data we can use for some type of mapping or other purpose.

Here's a rundown of how we'll solve this problem:
- Use a library called `geopy` to run an address through Google's geocoding service
- Use a sleep function to pause our script so we don't swamp Google and end up locked out
- Keep track of all of our data — coming in and going out post-geocode — with a keyed dictionary
- Put the original information AND our returned latitude, longitude and the match address (to check on the quality of the geocode) into a new CSV file

The first thing we'll need to do is import the Google geocoder from `geopy` as well as Python's `csv` and `time` libaries.

In [1]:
from geopy.geocoders import GoogleV3
import csv
import time

We've made objects already (think `BeautifulSoup()` and `csv.reader()`) that have allowed us to do things like read CSV files from our computers and parse HTML documents to isolate specific tags. In this case, we need to make a geolocator object that we can use to send our addresses to Google for the geocoding process.

In [2]:
geolocator = GoogleV3()

Let's open our address file and start a `DictReader` object that will give each data element in each row a key/value pair based on the header columns in the file. It follows the same process as creating a regular reader with the `csv` library.

In [3]:
address_file = open('payday_lenders.csv', 'rb')
input_file = csv.DictReader(address_file)

Set up a new file for our eventual output after the geocoding process is finished.

In [4]:
geocoded_file = open('payday_geocoded.csv', 'wb')

Here's where things change a bit. The CSV reader/writer we've been using doesn't care a ton about structure; it just wants a list of stuff to write as a row of data to the file. It doesn't matter how long the list is or anything like that, and we've been guiding the structure anyway to make everything's in its place.

Doing this with a dictionary instead of a list means that we have to adhere to a structure we set up. For the `DictWriter`, we have to give it a list of fields from the get-go to establish the order; we'll go ahead and write a header to the file, too.

In [5]:
output_fields = ['NAME', 'DBA', 'STADDR', 'STADDR2', 'CITY', 'STATE', 'ZIP', 'MATCH_ADDR', 'LAT_Y', 'LONG_X']
output = csv.DictWriter(geocoded_file, output_fields)
output.writeheader()

This loop should be familiar to you having written a couple that rely on writing rows a lists. The central difference here is the organization process — we'll modify the dictionary in each `row` variable as it comes in from the file, adding values and keys for latitude, longitude and the quality of the matched address before writing the original and this new information to the file.

We're going to put an if/else here to prevent the whole class from launching a volley of 500 requests at Google. Let's get the first five (row 1 is the header); we can check to see which line number in a file we're on by checking the `.line_num`.

Let's walk through the things we'll need this loop to do: 
- It needs to take values from our dictionary and assemble them into an address string that Google can actually decipher, something that takes a form like _"ADDRESS, CITY, STATE ZIP."_ In some cases we have addresses split across two separate fields, so we need to combine them and strip any unnecessary spaces before feeding them into the rest of the string.
- We need to take that string and geocode it.
- We need to plug results from the geocoder right back into the same `row` of data as new key/value pairs: the returned `latitude`, `longitude` and `address` Google was able to match on.
- Write the modified row to our new CSV file.
- To keep tabs on what's happening, we'll print ourselves a message with address and line number.
- **IMPORTANT:** Before we move on to the next row, pause for two seconds.

In [6]:
for row in input_file:
    
    if input_file.line_num <= 6:

        addr = (row['STADDR']+' '+row['STADDR2']).strip()+', '+row['CITY']+', '+row['STATE']+' '+row['ZIP']

        location = geolocator.geocode(addr)
        
        row['LAT_Y'] = location.latitude
        row['LONG_X'] = location.longitude
        row['MATCH_ADDR'] = location.address

        output.writerow(row)

        print('Attempted geocode of {0}, row {1}.'.format(addr, input_file.line_num))

        time.sleep(2)
    else:
        print('All done!')
        break

Attempted geocode of 8983  Potter Road, Des Plaines, IL 60016, row 2.
Attempted geocode of 320 West Illinois Route 173, Suite B, Antioch, IL 60002, row 3.
Attempted geocode of 1251D North Skokie Highway, Lake Bluff, IL 60044, row 4.
Attempted geocode of 8261 West Belmont Avenue, River Grove, IL 60171, row 5.
Attempted geocode of 19 Pullen Road, Metropolis, IL 62960, row 6.
All done!


Now that we're done, close both files so that everything's written to disk.

In [7]:
address_file.close()
geocoded_file.close()